In [1]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# pre-procesado
from tqdm import tqdm_notebook as tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Aprendizaje por transferencia traemos la red MobileNetV2
from tensorflow.keras.applications import MobileNetV2

# Capas
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model

# Optimizador
from tensorflow.keras.optimizers import RMSprop

%matplotlib inline
tf.__version__


Bad key "text.kerning_factor" on line 4 in
C:\Users\rafra\.conda\envs\tensorflow\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


'2.1.0'

### Descomprimir el dataset

In [2]:
# Data Path
dataset_path = "../input/cats_and_dogs_filtered.zip"

In [3]:
# Recognize as a zip file
#zip_obj = zipfile.ZipFile(file=dataset_path, mode='r')

In [4]:
#zip_obj.extractall("../input/")

In [5]:
#zip_obj.close()

### Configurar las rutas al dataset

In [6]:
dataset_path_new = '../input/cats_and_dogs_filtered/'

In [7]:
train_dir = os.path.join(dataset_path_new, 'train')
validation_dir = os.path.join(dataset_path_new, 'validation')

## Construir el Modelo

#### Cargar modelo pre entrenado (MobileNetV2)
    Cargaremos un modelo preentrenado por google llamado MobileNetV2

In [8]:
IMG_SHAPE = (128, 128, 3)

**Parametros**

    * include_top : no incluye la fase final de la red neuronal entrenada, por tanto creare una cabezera personalizada indicando el tipo de clasificación que yo deseo.
    * weights : elegimos 'imagenet' que es uno de los set de datos standar que utilizó para entrenar.

In [9]:
base_model = MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')

In [10]:
#base_model.summary()

#### Congelar modelo base

In [11]:
# Se congela la base que lleva la predicción en orden de personalizar nuetro modelo
base_model.trainable = False

#### Definir la cabecera personalizada para nuestra red neuronal

In [12]:
# podemos ver la capa de salida
base_model.output

<tf.Tensor 'out_relu/Identity:0' shape=(None, 4, 4, 1280) dtype=float32>

In [13]:
# GlobalAveragePooling2D -> Capa Operación de agrupación promedio global para datos temporales.
# De las 4X4 direcciones vamos a promediarlas
global_average_layer = GlobalAveragePooling2D()(base_model.output)

In [14]:
global_average_layer

<tf.Tensor 'global_average_pooling2d/Identity:0' shape=(None, 1280) dtype=float32>

In [15]:
# Conectar los 1280 valores  a una unica unidad que sera una probabilidad  de una de nuestras clases
prediction_layer = Dense(units=1, activation='sigmoid')(global_average_layer)

### Definir Modelo

In [16]:
# Combinación de las dos redes neuronales & definción del modelo
model = Model(inputs = base_model.input, outputs = prediction_layer)
#model.summary()

### Compilar el Modelo

In [17]:
# Recordar que es un problema de 
model.compile(optimizer=RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

### Crear Generadores de Datos

**Redimensionar imágenes**

    Las grandes arquitecturas pre-entrenadas solamente soportan cierto tipo de tamaños de imágenes.
    
    Por ejemplo: MobileNet (la arquitectura que nosotros usamos) soporta: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

In [18]:
# Generate batches of tensor image data with real-time data augmentation.
# The data will be looped over (in batches).
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [19]:
# carga las imagenes sin aplanarlas
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [20]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 1000 images belonging to 2 classes.


### Entrenar el modelo

In [22]:
model.fit(train_generator, epochs=8, validation_data=valid_generator)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 16 steps, validate for 8 steps
Epoch 1/8
16/16 [==============================] - 98s 6s/step - loss: 0.6671 - accuracy: 0.6120 - val_loss: 0.8103 - val_accuracy: 0.5350
Epoch 2/8
16/16 [==============================] - 101s 6s/step - loss: 0.6093 - accuracy: 0.6610 - val_loss: 0.7175 - val_accuracy: 0.5790
Epoch 3/8
16/16 [==============================] - 101s 6s/step - loss: 0.5618 - accuracy: 0.7065 - val_loss: 0.6415 - val_accuracy: 0.6240
Epoch 4/8
16/16 [==============================] - 101s 6s/step - loss: 0.5205 - accuracy: 0.7445 - val_loss: 0.5772 - val_accuracy: 0.6970
Epoch 5/8
16/16 [==============================] - 90s 6s/step - loss: 0.4835 - accuracy: 0.7765 - val_loss: 0.5178 - val_accuracy: 0.7600
Epoch 6/8
16/16 [==============================] - 95s 6s/step - loss: 0.4504 - accuracy: 0.8060 - val_loss: 0.4638 - val_accuracy: 0.8040
Epoch 7/8
16/16 [==============================] - 105s 7s/step - loss: 

### Evaluar el modelo de aprendizaje por transferencia

In [24]:
valid_loss, valid_accuracy = model.evaluate(valid_generator)

  ...
    to  
  ['...']
8/8 [==============================] - 10s 1s/step - loss: 0.3874 - accuracy: 0.8550


In [25]:
print("Accuracy afted transfer learning: {}".format(valid_accuracy))

Accuracy afted transfer learning: 0.8550000190734863


# Fine tuning : Puesta a punto de Parametros

Un par de cosas:

    **NUNCA HAY QUE USAR** la puesta a punto (fine tuning) de parámetros en toda la red neuronal: con algunas de las capas superiores (las finales) es más que suficiente suficiente. En la mayoría de casos, son las más especializadas. El objetivo del fine tuning es adaptar esa parte específica de la red neuronal para nuestro nuevo dataset específico.
    **Empezar con la puesta a punto DESPUÉS de haber finalizado la fase de aprendizaje** por transferencia. Si intentamos hacer el Fine tuning inmediatamente, los gradientes serán muy diferentes entre nuestra cabecera personalizada de la red neuronal y las nuevas capas no congeladas del modelo base.


In [26]:
base_model.trainable = True

In [27]:
print("Number of layers in the base model: {}".format(len(base_model.layers)))

Number of layers in the base model: 155


In [28]:
# vamos a congelar desde la primera hasta la numero 100
fine_tune_at = 100

In [29]:
# por medio de este bucle
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

### Compilar el modelo

In [30]:
model.compile(optimizer=RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

### Fine tuning

In [31]:
model.fit(train_generator, epochs=10, validation_data=valid_generator)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 16 steps, validate for 8 steps
Epoch 1/10
16/16 [==============================] - 115s 7s/step - loss: 0.1554 - accuracy: 0.9435 - val_loss: 0.0895 - val_accuracy: 0.9650
Epoch 2/10
16/16 [==============================] - 114s 7s/step - loss: 0.0148 - accuracy: 1.0000 - val_loss: 0.1095 - val_accuracy: 0.9630
Epoch 3/10
16/16 [==============================] - 117s 7s/step - loss: 0.0056 - accuracy: 1.0000 - val_loss: 0.1180 - val_accuracy: 0.9630
Epoch 4/10
16/16 [==============================] - 101s 6s/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.1356 - val_accuracy: 0.9620
Epoch 5/10
16/16 [==============================] - 102s 6s/step - loss: 8.0328e-04 - accuracy: 1.0000 - val_loss: 0.1470 - val_accuracy: 0.9610
Epoch 6/10
16/16 [==============================] - 101s 6s/step - loss: 3.0395e-04 - accuracy: 1.0000 - val_loss: 0.1671 - val_accuracy: 0.9610
Epoch 7/10
16/16 [==============================] - 102

## Evaluar el modelo red-calibrado

In [32]:
valid_loss, valid_accuracy = model.evaluate(valid_generator)

  ...
    to  
  ['...']
8/8 [==============================] - 10s 1s/step - loss: 0.2522 - accuracy: 0.9600


In [33]:
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

Validation accuracy after fine tuning: 0.9599999785423279
